In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from game import Game
from player import Player

In [2]:
from nazwisko import Nazwisko
from simplePlayer import SimplePlayer
from simpleNoCheck import SimpleNoCheck
from simpleAlwaysCheck import SimpleAlwaysCheck
from drawPlayer import DrawPlayer
from MyFirstPlayer import MyFirstPlayer
from MySecondPlayer import MySecondPlayer

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(9, 3), (14, 1), (10, 1), (13, 0), (9, 2), (14, 0), (13, 3), (10, 3)]
[(9, 0), (11, 0), (11, 3), (12, 3), (11, 2), (12, 2), (13, 2), (9, 1)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(results[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                # print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Nazwisko", "Nazwisko", Nazwisko],
            ["Kot1", "Kot1", MyFirstPlayer]
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 Nazwisko
2 Kot1
==== 1 Nazwisko
2 Kot1
==== 2 Kot1


In [9]:
full_results

[[None,
  [[1000, 0],
   [0, 0],
   [8.0, 7.521],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.521],
   [2.657, 0.0],
   [0, 0],
   0,
   0],
  [[53, 947],
   [0, 0],
   [16.507, 16.966],
   [12.933, 0.321],
   2.746,
   [8.205, 3.857],
   [0.031, 0.611],
   [9.423, 2.914],
   [0, 0],
   0,
   0]],
 [None,
  None,
  [[0, 1000],
   [0, 0],
   [7.488, 8.0],
   [15.0, 0.0],
   1.0,
   [0.0, 0.0],
   [7.488, 0.0],
   [0.0, 0.0],
   [0, 0],
   0,
   0]],
 [None, None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM Nazwisko Kot1 
0
RANDOM - 1000 53 
1
Nazwisko 0 - 0 
2
Kot1 947 1000 - 
Draws
 RANDOM Nazwisko Kot1 
0
RANDOM - 0 0 
1
Nazwisko 0 - 0 
2
Kot1 0 0 - 
Moves
 RANDOM Nazwisko Kot1 
0
RANDOM - 8.0 16.507 
1
Nazwisko 7.521 - 7.488 
2
Kot1 16.966 8.0 - 
Cards
 RANDOM Nazwisko Kot1 
0
RANDOM - 0.0 12.933 
1
Nazwisko 15.0 - 15.0 
2
Kot1 0.321 0.0 - 
Checks
 RANDOM Nazwisko Kot1 
0
RANDOM - 0.0 8.205 
1
Nazwisko 0.0 - 0.0 
2
Kot1 3.857 0.0 - 
Draw Decisions
 RANDOM Nazwisko Kot1 
0
RANDOM - 0.0 0.031 
1
Nazwisko 7.521 - 7.488 
2
Kot1 0.611 0.0 - 
Cheats
 RANDOM Nazwisko Kot1 
0
RANDOM - 2.657 9.423 
1
Nazwisko 0.0 - 0.0 
2
Kot1 2.914 0.0 - 
Errors
 RANDOM Nazwisko Kot1 
0
RANDOM - 0 0 
1
Nazwisko 0 - 0 
2
Kot1 0 0 - 


# to już dodałem od siebie, tego nie ma w komparatorze!!!!
tu jest fragment kodu który zrobi ranking, na podstawie czysto liczby zwycięstw

In [11]:
strategy = [
            ["Kot_First", "Kot_First", MyFirstPlayer],
            # ["RandomPlayer", "RANDOM", RandomPlayer],
            # # # # # # # # ["Nazwisko", "Nazwisko", Nazwisko], # ten jest zaimportowany, ale jest pusty praktycznie, jak drawPlayer
            ["SimplePlayer", "SimplePlayer", SimplePlayer],
            ["NoCheck", "NoCheck", SimpleNoCheck],
            ["AlwaysCheck", "AlwaysCheck", SimpleAlwaysCheck],
            # # # # # # # # ["DrawPlayer", "DrawPlayer", DrawPlayer], # no dobra, on jest zbyt chujowy by go brać pod uwagę ngl
            ["Kot_Second", "Kot_Second", MySecondPlayer],
           ]

#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    # print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        # print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results


def generate_ranking(full_results, strategy):
    n = len(strategy)
    total_wins = [0 for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                total_wins[i] += wins_i
                total_wins[j] += wins_j

    # Sparuj nazwy z wynikami
    ranking = list(zip([s[0] for s in strategy], total_wins))

    # Posortuj malejąco
    ranking.sort(key=lambda x: x[1], reverse=True)

    print("RANKING GRACZY (według liczby wygranych):")
    for idx, (name, wins) in enumerate(ranking, 1):
        print(f"{idx}. {name} - {wins} wygranych")

    return ranking

ranking = generate_ranking(full_results, strategy)

RANKING GRACZY (według liczby wygranych):
1. Kot_Second - 3190 wygranych
2. SimplePlayer - 2276 wygranych
3. Kot_First - 2258 wygranych
4. NoCheck - 2178 wygranych
5. AlwaysCheck - 98 wygranych


In [12]:
import pandas as pd
import numpy as np

def create_win_matrix(full_results, strategy):
    names = [s[0] for s in strategy]
    n = len(names)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = np.nan  # Brak pojedynków z samym sobą
            elif i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                matrix[i][j] = wins_i
                matrix[j][i] = wins_j

    df = pd.DataFrame(matrix, index=names, columns=names)
    print("MACIERZ WYGRANYCH:")
    print(df.round(1))

    return df


df = create_win_matrix(full_results, strategy)

MACIERZ WYGRANYCH:
              Kot_First  SimplePlayer  NoCheck  AlwaysCheck  Kot_Second
Kot_First           NaN         691.0    337.0       1000.0       230.0
SimplePlayer      309.0           NaN    661.0        991.0       315.0
NoCheck           663.0         339.0      NaN        922.0       254.0
AlwaysCheck         0.0           9.0     78.0          NaN        11.0
Kot_Second        770.0         685.0    746.0        989.0         NaN
